# <span style="color:#ff5f27"> 👨🏻‍🏫 Sklearn Transformation Functions Registration</span>

In this tutorial you will learn how to **register sklearn.pipeline with transformation functions and classifier** in Hopsworks Model Registry and use it in **training and inference pipelines**. 

## <span style="color:#ff5f27">🗄️ Table of Contents</span>
- [📝 Imports](#1)
- [⛳️ Feature Pipeline](#t1)
    - [💽 Loading Data](#2)
    - [🔮 Connecting to Hopsworks Feature Store](#3)
    - [🪄 Creating Feature Groups](#4)
- [⛳️ Training Pipeline](#t2)
    - [🖍 Feature View Creation](#5)
    - [👩🏻‍🔬 Data Transformation Pipeline](#6)
    - [🧬 Modeling](#7)
    - [💾 Saving the Model in the Model Registry](#8)
- [⛳️ Inference Pipeline](#t3)
    - [📮 Retrieving the Model from the Model Registry](#9)
    - [👨🏻‍⚖️ Batch Prediction](#10)
    - [👨🏻‍⚖️ Serving Feature Vector Prediction](#11)

<a name='1'></a>
## <span style='color:#ff5f27'> 📝 Imports </span>

In [ ]:
# Importing the required libraries
import pandas as pd          # Pandas for data manipulation
import numpy as np           # NumPy for numerical operations
import os                    # os for interacting with the operating system
import joblib                # joblib for saving and loading models

# Importing classes and functions from scikit-learn for data preprocessing and modeling
from sklearn.compose import ColumnTransformer   # For transforming specific columns in a DataFrame
from sklearn.pipeline import Pipeline           # For creating a data processing pipeline
from sklearn.impute import SimpleImputer        # For handling missing values
from sklearn.preprocessing import StandardScaler, OneHotEncoder   # For feature scaling and one-hot encoding
from sklearn.feature_selection import SelectPercentile, chi2    # For feature selection using chi-squared test

# Importing XGBoost library for gradient boosting-based machine learning
import xgboost as xgb

# Importing a metric to evaluate the model's performance
from sklearn.metrics import accuracy_score      # For measuring classification accuracy

---
<a name='t1'></a>
# <span style="color:#ff5f27;">⛳️ Feature Pipeline </span>

In this section you will load data, create a Hopsworks feature group and insert your dataset into created feature group.

<a name='2'></a>
## <span style="color:#ff5f27;"> 💽 Loading Data </span>

To begin with, let's load a dataset which contains air quality measurements for different  cities from 2013-01-01 to 2023-04-11.

In [ ]:
# Load the data
df_original = pd.read_csv("https://repo.hops.works/dev/davit/air_quality/backfill_pm2_5_eu.csv")
df_original.head(3)

After loading the data, let's add a new column called 'target' to the DataFrame. 

The purpose of this column is to serve as the target variable for a binary classification task. In binary classification, we aim to predict one of two possible outcomes (in this case, either 0 or 1) based on the input features (other columns in the DataFrame).

To create the **target** column, you will use the `np.random.choice` function from the NumPy library. It randomly assigns either a 0 or a 1 to each row in the DataFrame. 

This randomness helps simulate a scenario where we don't have the actual target values yet, so we create a placeholder **for demonstration purposes**.

In [ ]:
# Generate a binary target column
df_original['target'] = np.random.choice([0, 1], size=len(df_original))
df_original.head(3)

<a name='3'></a>
## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

The next step is to login to the Hopsworks platform. 

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

<a name='4'></a>
## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

Now you need to create a Feature Group and insert your dataset.

You will use `.get_or_create_feature_group()` method of the feature store object with the next parameters:

- name='feature_group_online': This specifies the name of the feature group as 'feature_group_online'.

- description='Online Feature Group': A brief description provided for the feature group.

- version=1: The version number of the feature group.

- primary_key=['city_name', 'date']: The primary key of the feature group. It indicates which columns in the DataFrame uniquely identify each record.

- online_enabled=True: This parameter enables the feature group for online serving, meaning it can be used in real-time for making predictions.

The `feature_group.insert(df_original)` statement inserts the `df_original` DataFrame into the feature group.

In [ ]:
feature_group = fs.get_or_create_feature_group(
    name='feature_group_online',
    description='Online Feature Group',
    version=1,
    primary_key=['city_name', 'date'],
    online_enabled=True,
)    
feature_group.insert(df_original)

---
<a name='t2'></a>
# <span style="color:#ff5f27;">⛳️ Training Pipeline </span>

In the **Training Pipeline** you will create a train-test split, define data preprocessing steps and put then into sklearn.pipeline with the XGBClassifier. Then you will register your sklearn.pipeline into Hopsworks Model Registry.

<a name='5'></a>
## <span style="color:#ff5f27;"> 🖍 Feature View Creation</span>

In this part you will build a Query object and create a feature view.


In Hopsworks Feature Store, a Query object allows you to select specific features from a feature group.

`feature_group.select_except(['date'])` selects all columns from the feature group except for the 'date' column. It means that the 'date' column will be excluded from the Query object.

In [ ]:
# Create a Query object
query = feature_group.select_except(['date'])
query.show(3)

After creating the Query object, you will create a feature view.

A feature view is a logical representation of data which can be used for real-time serving or batch processing. 

The `.get_or_create_feature_view()` method has the next parameters:

- name='serving_fv': This specifies the name of the feature view.

- version=1: The version number of the feature view.

- query=query: The Query object created earlier is provided as input to the feature view. It determines the set of required features.

- labels=['target']: This parameter defines the target column name.

In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='serving_fv',
    version=1,
    query=query,
    labels=['target'],
)

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>
The next step is to create the train-test split of your data.

Let's clarify the next parameters of the `.create_train_test_split()` method:

- test_size=0.1: This parameter specifies the size of the test set relative to the entire dataset. In this case, the test set will contain 10% of the data, and the train set will have the remaining 90%.

- description='Description of the dataset': A brief description provided for the train-test split dataset, explaining its purpose or any other relevant information.

- data_format='csv': This parameter specifies the format in which the train-test split dataset will be stored. Here, it is set to 'csv', meaning the dataset will be saved in CSV format.

In [ ]:
# Create a train-test split dataset
td_version, job = feature_view.create_train_test_split(
    test_size=0.1,
    description='Description of the dataset',
    data_format='csv'
)

### <span style="color:#ff5f27;">🪝 Training Dataset Retrieval</span>

To retrieve your train_test_split you can use the `.get_train_test_split()` method of the feature_view object.

The parameter `training_dataset_version` specifies the version number of the train-test split dataset to retrieve. 

`td_version` is the version number that was obtained when the train-test split dataset was created in a previous step.

In [ ]:
# Retrieve the train-test split
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(
    training_dataset_version=td_version
)

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

<a name='6'></a>
## <span style="color:#ff5f27;">👩🏻‍🔬 Data Transformation Pipeline</span>

The next step is to build a Data Transformation Pipeline.

A pipeline allows you to chain multiple data preprocessing steps and machine learning model into a single object. This is particularly useful for automating and simplifying the workflow in a machine learning project, as it ensures that all the necessary steps are applied consistently to both the training and testing data.

Let's define transformation steps for numeric and categorical features and put then into pipelines.

In [ ]:
# Define a Pipeline for numeric features
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)
# Define a Pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)

- `SimpleImputer transformer` is used to handle missing values in numeric features. The strategy chosen is "median," meaning that missing values will be replaced with the median of the existing values in that feature.

- `StandardScaler` is used to standardize the numeric features so that they have a mean of 0 and a standard deviation of 1. Standardization is essential to ensure that features are on similar scales, which can be beneficial for certain machine learning algorithms.

- `OneHotEncoder` is used to convert categorical features into a binary representation, where each category is transformed into a binary vector (0s and 1s). The **handle_unknown="ignore"** parameter allows the encoder to ignore unknown categories during transformation, avoiding errors when encountering unseen categories during inference.

- `SelectPercentile` is used to select the most informative features from the encoded binary vectors based on their chi-squared statistics. The parameter percentile=50 indicates that the top 50% most informative features will be retained, and the rest will be discarded.

Then you need to use `ColumnTransformer` to combine several transformers into a single transformer. 

It allows you to specify different preprocessing steps for specific subsets of columns of the input data.

In [ ]:
# Define a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, ['pm2_5']),
        ("cat", categorical_transformer, ['city_name']),
    ]
)

<a name='7'></a>
## <span style="color:#ff5f27;">🧬 Model Fit</span>

Now let's combine the `preprocessor` and `XGBClassifier` into one **pipeline** and fit it to the training data.

In [ ]:
# Create a pipeline
xgb_classifier = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", xgb.XGBClassifier())
    ]
)
# Fit the classifier
xgb_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("👮🏻‍♂️ Accuracy:", accuracy)

## <span style="color:#ff5f27;">🗄 Model Registry</span>

In Hopsworks, the Model Registry is a crucial component used to manage and version machine learning models. It acts as a centralized repository where trained models can be stored, tracked, and shared among team members.

By calling `project.get_model_registry()`, the code retrieves a reference to the Model Registry associated with the current Hopsworks project. This reference allows the user to interact with the Model Registry and perform operations such as registering, versioning, and accessing trained machine learning models.
With the Model Registry, data scientists and machine learning engineers can effectively collaborate, track model changes, and easily deploy the best-performing models to production environments.

In [ ]:
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The next step is to **define input and output schema** of a machine learning model.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train.values)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

<a name='8'></a>
### <span style="color:#ff5f27;">💾 Saving the Model</span>

Now you are ready to register your model in the Hopsworks Moder Registry.

To begin with, let's create the `xgb_tf_model` model directory and save the trained model in this directory.

In [ ]:
model_dir = "xgb_tf_model"

if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(xgb_classifier, model_dir + '/xgb_classifier.pkl')

To register your model in the Hopsworks model registry you can use `.create_model()` method with the next parameters:

- name="xgb_model": The name of the model.

- metrics={"Accuracy": accuracy}: The model's performance metrics are specified as a dictionary, with "Accuracy" as the key and the value being the accuracy score computed earlier in the code. This metric represents the accuracy of the model's predictions on the test data.

- description="XGB model": A brief description of the model.

- input_example=X_train.sample(): An example input from the training data (X_train) is used to demonstrate the expected format of the model's input data. It is randomly sampled from X_train.

- model_schema=model_schema: The model schema, which represents the data input and output structure of the model, is specified using the previously defined model_schema.

In [ ]:
# Create a model in the model registry
model = mr.python.create_model(
    name="xgb_model",
    metrics={"Accuracy": accuracy}, 
    description="XGB model",
    input_example=X_train.sample(),
    model_schema=model_schema
)
model.save(model_dir)

---
<a name='t3'></a>
# <span style="color:#ff5f27;">⛳️ Inference Pipeline </span>

In the **Inference Pipeline** section, you will retrieve your model from Hopsworks Model Registry and utilize this model to make predictions on both Batch Data and Serving Feature Vectors.

<a name='9'></a>
## <span style="color:#ff5f27;"> 📮 Retrieving the Model from Model Registry </span>

To retrieve a previously registered machine learning model from the Hopsworks Model Registry you need to use the `.get_model()` method with the next parameters:

- name="xgb_model": The name of the model to be retrieved.

- version=1: The version number of the model to be retrieved.

Then you will download the model from the Model Registry.

In [ ]:
# Retrieve your model from the model registry
retrieved_model = mr.get_model(
    name="xgb_model",
    version=1
)
saved_model_dir = retrieved_model.download()

In [ ]:
retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgb_classifier.pkl")
retrieved_xgboost_model

<a name='10'></a>
## <span style="color:#ff5f27;"> 👨🏻‍⚖️ Batch Prediction </span>

Batch prediction is a process in which a trained machine learning model is used to make predictions on a large set of data all at once.

It's important to note that while batch prediction is efficient for large-scale data processing, it may not be suitable for applications that require immediate, real-time predictions, such as online recommendations or fraud detection systems.

To retrieve batch data from the feature view you need to use `init_batch_scoring` method of the feature view object.

`training_dataset_version` parameter specifies the version number of the training dataset that will be used for scoring.

Then you can use the `.get_batch_data()` method to retrieve batch data.

In [ ]:
# Initialise feature view to retrieve batch data
feature_view.init_batch_scoring(training_dataset_version=td_version)

# Retrieve batch data
batch_data = feature_view.get_batch_data()
batch_data.head(3)

Now let's use retrieved model to predict batch data.

In [ ]:
# Predict batch data using retrieved model
predictions_batch = retrieved_xgboost_model.predict(batch_data)
predictions_batch[:10]

<a name='11'></a>
## <span style="color:#ff5f27;"> 👨🏻‍⚖️ Serving Feature Vector Prediction</span>

**Serving Feature Vector Prediction** is a process of using a trained machine learning model to make predictions on feature vector(s) in real-time. 

In contrast to batch prediction, where predictions are made on a large set of data simultaneously, serving feature vector prediction focuses on making predictions on-demand, usually as new data points arrive or as users interact with a system.

To begin with, let's create `to_df` function which will transform a feature vector(s) list into a pandas DataFrame.

In [ ]:
def to_df(feature_vector):
    """
    Convert a feature vector or a list of feature vectors into a pandas DataFrame.

    Parameters:
        feature_vector (a list, or list of lists): 
            A feature vector or a list of feature vectors. A feature vector is 
            represented as a list containing two elements: the first 
            element corresponds to the city name (categorical feature), and the 
            second element corresponds to the PM2.5 value (numerical feature).

    Returns:
        pandas.DataFrame: A DataFrame representing the feature vector(s). 
        The DataFrame will have two columns: 'city_name' for the city names 
        and 'pm2_5' for the corresponding PM2.5 values.

    Example:
        >>> feature_vector = ['New York', 15.3]
        >>> to_df(feature_vector)
           city_name  pm2_5
        0  New York   15.3

        >>> multiple_vectors = [['New York', 15.3], ['Los Angeles', 10.7]]
        >>> to_df(multiple_vectors)
          city_name  pm2_5
        0  New York   15.3
        1  Los Angeles 10.7
    """
    
    # Check if the input is a list of feature vectors
    if isinstance(feature_vector[0], list): 
        # Separate the city names and PM2.5 values into separate lists
        city_names = [vector[0] for vector in feature_vector]
        pm2_5_values = [vector[1] for vector in feature_vector]
        
        # Create a DataFrame with 'city_name' and 'pm2_5' columns from the lists
        data = pd.DataFrame(
            {
                'city_name': city_names,
                'pm2_5': pm2_5_values,
            }
        )
        
        # Return the DataFrame representing multiple feature vectors
        return data

    # If only one feature vector is provided, create a DataFrame for it
    data = pd.DataFrame(
            {
                'city_name': [feature_vector[0]],
                'pm2_5': [feature_vector[1]],
            }
        )
    
    # Return the DataFrame representing a single feature vector
    return data

The next step is to initialize the feature view for serving and then retrieve a feature vector with specified primary keys.

In [ ]:
# Initialise feature view to retrieve feature vector
feature_view.init_serving(1)

# Retrieve a feature vector
feature_vector = feature_view.get_feature_vector(
    entry = {
        "city_name": 'Amsterdam',
        "date": '2013-01-01',
    }
)
feature_vector

Let's apply `to_df` function in order to transform the feature vector into pandas dataframe.

In [ ]:
# Transform feature vector to pandas dataframe
feature_vector_df = to_df(feature_vector)
feature_vector_df

Now you can use your model to predict the feature vector dataframe.

In [ ]:
# Predict transformed feature vector using retrieved model
prediction_feature_vector = retrieved_xgboost_model.predict(feature_vector_df)
prediction_feature_vector

In addition, you can retrieve several feature vectors. Just pass primary keys as a list of dictionaries.

In [ ]:
# Retrieve feature vectors from feature store
feature_vectors = feature_view.get_feature_vectors(
    entry = [
        {"city_name": 'Amsterdam', "date": '2013-01-01'},
        {"city_name": 'Amsterdam', "date": '2014-01-01'},
    ]
)
feature_vectors

Apply `to_df` function in order to transform feature vectors into pandas dataframe.

In [ ]:
# Convert feature vectors to pandas dataframe
feature_vectors_df = to_df(feature_vectors)
feature_vectors_df

Now you can use your model to predict the dataframe which contains feature vectors.

In [ ]:
# Predict transformed feature vectors using retrieved model
prediction_feature_vectors = retrieved_xgboost_model.predict(feature_vectors_df)
prediction_feature_vectors

---